In [31]:
import json
import os

class Song:
    def __init__(self,key,scale,tonic,chords,meter_beat,numBeats,beatUnit,notes, **kwargs):
        # self.absolute_root = self.note_to_midi(key)
        self.key = key
        self.scale = scale
        self.tonic = tonic
        self.meter_beat = meter_beat
        self.numBeats = numBeats
        self.beatUnit = beatUnit
        self.allowed_keys_list = self.generated_keys_list()
        print(self.allowed_keys_list)
        self.chords = chords
        self.notes = notes
        self.generated_absolute_chords()
        self.generated_absolute_notes()
    def generated_keys_list(self):
        oct_start = 0
        oct_end = 10
        get_key_shift = self.get_key_shift(self.scale)
        absolute_root = self.note_to_midi(self.tonic, octave=oct_start)
        first_octave = [x + absolute_root for x in get_key_shift]
        key_list = [x + (12 * i) for i in range(oct_end) for x in first_octave]
        return key_list
    def generated_absolute_notes(self):
        if len(self.notes) == 0: return 
        for c in self.notes:
            print(c.isRest)
            if not c.isRest:
                print("~~~~~")
                absolute_note = self.sd_to_midi(c.sd,c.octave) 
                print(absolute_note)
                c.absolute_note_position.append(absolute_note)
            
    def generated_absolute_chords(self):
        if len(self.chords) == 0: return 
        for c in self.chords:
            absolute_root = self.note_to_midi(self.tonic) 
            root_abs_position = self.allowed_keys_list.index(absolute_root) + c.root - 1 
            c.absolute_chord_position = [self.allowed_keys_list[root_abs_position],self.allowed_keys_list[root_abs_position+2],self.allowed_keys_list[root_abs_position+4]]
    def sd_to_midi(self, sd, octave):
        root_note = self.note_to_midi(self.tonic) 
        lowest_octave_root = (root_note % 12) + 60 
        root_note_abs_position = self.allowed_keys_list.index(lowest_octave_root)
        note =  self.allowed_keys_list[root_note_abs_position + int(sd) + (octave * 12)]
        return note
    def get_key_shift(self, mode):
        ks = {
            "major" : [0, 2, 4, 5, 7, 9, 11],
            "minor" : [0, 2, 3, 5, 7, 8, 10],
            "dorian" : [0, 2, 3, 5, 7, 9, 10],
            "locrian" : [0, 1, 3, 5, 6, 8, 10],
            "mixolydian" : [0, 2, 4, 5, 7, 9, 10],
            "harmonicMinor" : [0, 2, 3, 5, 7, 8, 11],
            "lydian" : [0, 2, 4, 6, 7, 9, 11],
            "phrygian" : [0, 1, 3, 5, 7, 8, 10],
            "phrygianDominant" : [0, 1, 3, 5, 7, 8, 9]
        }
        return ks[mode]
    def note_to_midi(self, note, octave=4):
        notes = {
            'C': 0,
            'Db': 1,
            'C#': 1,
            'D': 2,
            'Eb': 3,
            'D#': 3,
            'E': 4,
            'F': 5,
            'Gb': 6,
            'F#': 6,
            'G': 7,
            'Ab': 8,
            'G#': 8,
            'A': 9,
            'Bb': 10,
            'A#': 10,
            'B': 11
        }
        return notes[note] + (octave * 12)
    def __str__(self):
        return f"key: {self.scale} \t {self.tonic}"
    def __repr__(self):
        return f"key: {self.scale} \t {self.tonic}"


class Chord:
    def __init__(self, root, beat, duration, type, inversion, applied, adds, omits, alterations, suspensions, substitutions, pedal, alternate, borrowed, isRest, recordingEndBeat=None):
        self.root = root
        self.beat = beat
        self.duration = duration
        self.type = type
        self.inversion = inversion
        self.applied = applied
        self.adds = adds
        self.omits = omits
        self.alterations = alterations
        self.suspensions = suspensions
        self.substitutions = substitutions
        self.pedal = pedal
        self.alternate = alternate
        self.borrowed = borrowed
        self.isRest = isRest
        self.recordingEndBeat = recordingEndBeat
        self.absolute_chord_position = []
    def __repr__(self):
        roman_numerals = {1: 'I', 2: 'II', 3: 'III', 4: 'IV', 5: 'V', 6: 'VI', 7: 'VII'}
        roman = roman_numerals.get(self.root, '?')
        return f"({roman} : {self.beat}-{self.beat + self.duration - 1})"
    def __str__(self):
        roman_numerals = {1: 'I', 2: 'II', 3: 'III', 4: 'IV', 5: 'V', 6: 'VI', 7: 'VII'}
        roman = roman_numerals.get(self.root, '?')
        return f"({roman} : {self.beat}-{self.beat + self.duration - 1})"

class Note:
    def __init__(self, sd, octave, beat, duration, isRest=None, **kwargs):
        self.sd = sd
        self.octave = octave
        self.beat = beat
        self.duration = duration
        self.isRest = isRest
        self.absolute_note_position = []
    def __str__(self):
        return f"({self.sd}-{self.octave}: {self.beat}-{self.beat + self.duration - 0.5})"
    def __repr__(self):
        return f"({self.sd}-{self.octave}: {self.beat}-{self.beat + self.duration - 0.5})"



def load_music_data(json_file_path):
    try:
        with open(json_file_path, 'r', encoding='utf-8-sig') as file:
            data = json.load(file)
    except FileNotFoundError:
        print("Error: The file was not found.")
        return None, None
    except json.JSONDecodeError:
        print("Error: There was an issue decoding the JSON file.")
        return None, None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None, None

    chords = [Chord(**chord_data) for chord_data in data.get('chords', [])]
    notes = [Note(**note_data) for note_data in data.get('notes', [])]

    # Extract the first key (assuming there's only one)
    key = data.get('keys', [{}])[0]

    # Get the scale and tonic fields
    scale = key.get('scale')
    tonic = key.get('tonic')
    print(tonic, scale)
    # print(song)
    
    # meters = key.get('meters')
    meter = data.get('meters', [{}])[0]
    print(meter)
    # if meters:
    # meter = meters[0]  # get the first meter in the list
    meter_beat = meter.get('beat')
    numBeats = meter.get('numBeats')
    beatUnit = meter.get('beatUnit')

    song = Song(f"{tonic}-{scale}",scale,tonic,chords,meter_beat,numBeats,beatUnit,notes)

    return chords, notes, song


json_file_path = '/Users/Juan.Huerta/github/music_data/data/'
scale_tonic_counts = {}

for filename in os.listdir(json_file_path):
    if filename.endswith('.json'):
        file_path = os.path.join(json_file_path, filename)
        print(file_path)
        chords_list, notes_list, song = load_music_data("/Users/Juan.Huerta/github/music_data/data/A Team.json")
        if chords_list is not None and notes_list is not None:
            # scale = song.scale
            # tonic = song.tonic
            print(chords_list[0].root)
            print(song.chords[0].absolute_chord_position)
            
            print(chords_list[1].root)
            print(song.chords[1].absolute_chord_position)

            break
    break
            # if scale and tonic:
            #     pair = f"{tonic}-{scale}"
            #     scale_tonic_counts[pair] = scale_tonic_counts.get(pair, 0) + 1
        # if scale:
        #         pair = f"{scale}"
        #         scale_tonic_counts[scale] = scale_tonic_counts.get(scale, 0) + 1
        # if len(chords_list)>0:
        #     for c in chords_list:
        #         pair = f"{c}"
        #         scale_tonic_counts[c] = scale_tonic_counts.get(c, 0) + 1
        #         break

# print(scale_tonic_counts)


# # Example usage:
# json_file_path = '/Users/Juan.Huerta/github/music_data/bw.json'
# chords_list, notes_list, song = load_music_data(json_file_path)
# if chords_list is not None and notes_list is not None:
#     print("Chords and notes loaded successfully.")
# else:
#     print("Failed to load chords or notes.")


# print(chords_list)

/Users/Juan.Huerta/github/music_data/data/here comes the sun-2.json
A major
{'beat': 1, 'numBeats': 4, 'beatUnit': 1}
[9, 11, 13, 14, 16, 18, 20, 21, 23, 25, 26, 28, 30, 32, 33, 35, 37, 38, 40, 42, 44, 45, 47, 49, 50, 52, 54, 56, 57, 59, 61, 62, 64, 66, 68, 69, 71, 73, 74, 76, 78, 80, 81, 83, 85, 86, 88, 90, 92, 93, 95, 97, 98, 100, 102, 104, 105, 107, 109, 110, 112, 114, 116, 117, 119, 121, 122, 124, 126, 128]
True
False
~~~~~
52
False
~~~~~
54
True
False
~~~~~
40
False
~~~~~
50
True
False
~~~~~
52
False
~~~~~
54
False
~~~~~
54
False
~~~~~
57
False
~~~~~
54
True
False
~~~~~
52
False
~~~~~
54
True
False
~~~~~
37
False
~~~~~
37
True
1
[57, 61, 64]
5
[64, 68, 71]


In [42]:
import pretty_midi
import matplotlib.pyplot as plt
%matplotlib inline
# For putting audio in the notebook
import IPython.display


chords_list, notes_list, song = load_music_data("/Users/Juan.Huerta/github/music_data/data/A Team.json")


# Let's add a few notes to our instrument
pm = pretty_midi.PrettyMIDI(initial_tempo=200)
inst = pretty_midi.Instrument(program=42, is_drum=False, name='piano')
pm.instruments.append(inst)

# Define the BPM and tempo
bpm = 85  # beats per minute
tempo = 60 / bpm  # seconds per beat

velocity = 100
shift = 0

for c in song.chords:
    numBeats = 4  # c.duration
    duration = c.duration
    for i in range((duration + numBeats - 1) // numBeats):  # Ceiling division
        start = c.beat + i * numBeats
        end = min(start + numBeats, c.beat + duration)  # Don't exceed the original duration
        print(c.absolute_chord_position)
        for pitch in c.absolute_chord_position:
            start_time = start * tempo  # Convert beats to seconds
            end_time = end * tempo  # Convert beats to seconds
            inst.notes.append(pretty_midi.Note(velocity, pitch, start_time, end_time))
            
            
for c in song.notes:
    start = c.beat * tempo  # Convert beat to seconds
    end = (c.beat + c.duration) * tempo  # Convert beat + duration to seconds
    if len(c.absolute_note_position)>0:
        # print(c.absolute_note_position[0])
        inst.notes.append(pretty_midi.Note(velocity, c.absolute_note_position[0], start, end))
# Synthesis frequency
fs = 16000
IPython.display.Audio(pm.synthesize(fs=16000), rate=16000)

A major
{'beat': 1, 'numBeats': 4, 'beatUnit': 1}
[9, 11, 13, 14, 16, 18, 20, 21, 23, 25, 26, 28, 30, 32, 33, 35, 37, 38, 40, 42, 44, 45, 47, 49, 50, 52, 54, 56, 57, 59, 61, 62, 64, 66, 68, 69, 71, 73, 74, 76, 78, 80, 81, 83, 85, 86, 88, 90, 92, 93, 95, 97, 98, 100, 102, 104, 105, 107, 109, 110, 112, 114, 116, 117, 119, 121, 122, 124, 126, 128]
True
False
~~~~~
52
False
~~~~~
54
True
False
~~~~~
40
False
~~~~~
50
True
False
~~~~~
52
False
~~~~~
54
False
~~~~~
54
False
~~~~~
57
False
~~~~~
54
True
False
~~~~~
52
False
~~~~~
54
True
False
~~~~~
37
False
~~~~~
37
True
[57, 61, 64]
[57, 61, 64]
[64, 68, 71]
[66, 69, 73]
[62, 66, 69]
[57, 61, 64]


In [1]:
!pip install pretty_midi

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 37.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 219.4 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592287 sha256=06c2d811890712f387d8799b7978e81383a09b23a81f1350b914de95b69c1130
  Stored in directory: /private/var/folders/jc/x36d_6cx17gbsndvcn92rqdw0000gp/T/pip-ephem-wheel-cache-2rsyxmat/wheels/75/ec/20/b8e937a5bcf1de547ea5ce465db7de7f6761e15e6f0a01e25f
Successfully built pretty_midi


In [29]:
class KeyShifts:
    major = [2, 2, 1, 2, 2, 2, 1]
    minor = [2, 1, 2, 2, 1, 2, 2]
    dorian = [2, 1, 2, 2, 2, 1, 2]
    locrian = [1, 2, 2, 1, 2, 2, 2]
    mixolydian = [2, 2, 1, 2, 2, 1, 2]
    harmonicMinor = [2, 1, 2, 2, 1, 3, 1]
    lydian = [2, 2, 2, 1, 2, 2, 1]
    phrygian = [1, 2, 2, 2, 1, 2, 2]
    phrygianDominant = [1, 2, 2, 2, 1, 2, 1]
    


In [ ]:
    def get_key_shift(self, mode):
        ks = {
            "major" : [2, 2, 1, 2, 2, 2, 1],
            "minor" : [2, 1, 2, 2, 1, 2, 2],
            "dorian" : [2, 1, 2, 2, 2, 1, 2],
            "locrian" : [1, 2, 2, 1, 2, 2, 2],
            "mixolydian" : [2, 2, 1, 2, 2, 1, 2],
            "harmonicMinor" : [2, 1, 2, 2, 1, 3, 1],
            "lydian" : [2, 2, 2, 1, 2, 2, 1],
            "phrygian" : [1, 2, 2, 2, 1, 2, 2],
            "phrygianDominant" : [1, 2, 2, 2, 1, 2, 1]
        }
        return ks[mode]
        